In [1]:
import pandas as pd
import numpy as np
from PIL import Image
import os
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.amp import GradScaler, autocast
import cv2
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau

# --- Кастомный датасет ---
class HASYv2Dataset(Dataset):
    def __init__(self, df, images_dir, transform=None):
        self.df = df
        self.images_dir = images_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = os.path.basename(self.df.iloc[idx]['path'])
        img_path = os.path.join(self.images_dir, img_name)
        try:
            img = Image.open(img_path).convert('L')
        except FileNotFoundError:
            print(f"Файл не найден: {img_path}")
            raise
        label = self.df.iloc[idx]['normalized_symbol_id']
        if self.transform:
            img = self.transform(img)
        return img, label

# --- Улучшенная CNN ---
class EnhancedSymbolCNN(nn.Module):
    def __init__(self, num_classes=369):
        super(EnhancedSymbolCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 4 * 4, 1024),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# --- Тест форматирования ---
def test_formatting(dataset, output_dir='test_formatted'):
    os.makedirs(output_dir, exist_ok=True)
    for i in range(min(5, len(dataset))):
        # Получаем исходное изображение без трансформации
        img_name = os.path.basename(dataset.df.iloc[i]['path'])
        img_path = os.path.join(dataset.images_dir, img_name)
        img = Image.open(img_path).convert('L')
        label = dataset.df.iloc[i]['normalized_symbol_id']

        # Применяем бинаризацию и инверсию
        img_np = np.array(img)
        _, img_np = cv2.threshold(img_np, 128, 255, cv2.THRESH_BINARY)
        img_np = 255 - img_np  # Инверсия для черного фона и белого символа
        img_pil = Image.fromarray(img_np.astype(np.uint8))
        img_pil.save(os.path.join(output_dir, f'test_{i}_label_{label}.png'))
    print(f"✅ Тестовые изображения сохранены в {output_dir}")

# --- Основная функция обучения ---
def main():
    # Загрузка CSV
    csv_path = 'hasyv2/hasyv2/hasy-data-labels.csv'
    df = pd.read_csv(csv_path)

    # Нормализация symbol_id
    unique_symbol_ids = sorted(df['symbol_id'].unique())
    id_to_normalized = {old_id: new_id for new_id, old_id in enumerate(unique_symbol_ids)}
    if len(unique_symbol_ids) != 369:
        raise ValueError(f"Ожидалось 369 уникальных классов, найдено {len(unique_symbol_ids)}")
    df['normalized_symbol_id'] = df['symbol_id'].map(id_to_normalized)
    print(f"Нормализованный диапазон symbol_id: [{df['normalized_symbol_id'].min()}, {df['normalized_symbol_id'].max()}]")
    print(f"Количество уникальных классов: {len(df['normalized_symbol_id'].unique())}")

    # Обновление словаря class_to_symbol
    with open('hasyv2/hasyv2/symbols.csv', 'r', encoding='utf-8') as f:
        symbols_df = pd.read_csv(f)
    class_to_symbol = {}
    for index, row in symbols_df.iterrows():
        class_id = row['symbol_id']
        latex_command = row['latex']
        class_to_symbol[str(id_to_normalized[class_id])] = latex_command
    with open('model2/class_to_symbol_normalized.json', 'w', encoding='utf-8') as f:
        json.dump(class_to_symbol, f, ensure_ascii=False, indent=4)
    print("✅ Словарь сохранён в model/class_to_symbol_normalized.json")

    # Разделение данных
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    images_dir = 'hasyv2/hasyv2/hasy-data'

    # Аугментация данных с небелыми фонами
    train_transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.RandomInvert(p=0.5),
        transforms.RandomRotation(15),
        transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.8, 1.2)),
        transforms.RandomPerspective(distortion_scale=0.2, p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
        transforms.RandomErasing(p=0.5, scale=(0.02, 0.1)),
    ])
    test_transform = transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], std=[0.5]),
    ])

    # Создание датасетов
    train_dataset = HASYv2Dataset(train_df, images_dir, transform=train_transform)
    test_dataset = HASYv2Dataset(test_df, images_dir, transform=test_transform)

    # Тест форматирования
    test_formatting(train_dataset)

    # Вычисление классовых весов
    class_weights = compute_class_weight('balanced', classes=np.unique(df['normalized_symbol_id']),
                                        y=df['normalized_symbol_id'])
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    # Создание загрузчиков
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory=True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, pin_memory=True, num_workers=4)
    print(f"✅ Данные загружены: {len(train_dataset)} обучающих, {len(test_dataset)} тестовых образцов")

    # Инициализация модели
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = EnhancedSymbolCNN(num_classes=369).to(device)
    print(f"✅ Модель на {device}")

    # Загрузка предобученной модели, если существует
    model_path = 'model/hasyv2_model_arg.pth'
    if os.path.exists(model_path):
        model.load_state_dict(torch.load(model_path, map_location=device, weights_only=True))
        print(f"✅ Загружена предобученная модель из {model_path}")

    # Оптимизатор и функция потерь
    criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-3)
    scaler = GradScaler()
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=5, factor=0.5, verbose=True)

    # Обучение
    num_epochs = 50
    best_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch')
        for images, labels in progress_bar:
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item()
            progress_bar.set_postfix({'loss': f'{loss.item():.4f}'})
        avg_loss = running_loss / len(train_loader)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Average Loss: {avg_loss:.4f}')

        # Оценка
        model.eval()
        correct = 0
        total = 0
        top5_correct = 0
        with torch.no_grad():
            for images, labels in tqdm(test_loader, desc='Evaluating', unit='batch'):
                images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                with autocast(device_type='cuda' if torch.cuda.is_available() else 'cpu'):
                    outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                # Top-5 accuracy
                _, top5_pred = outputs.topk(5, dim=1)
                top5_correct += top5_pred.eq(labels.view(-1, 1).expand_as(top5_pred)).sum().item()
        accuracy = correct / total
        top5_accuracy = top5_correct / total
        print(f'Точность на тестовой выборке: {accuracy:.4f} ({correct}/{total}), Top-5 точность: {top5_accuracy:.4f}')

        # Обновление scheduler
        scheduler.step(avg_loss)

        # Сохранение модели после каждой эпохи
        torch.save(model.state_dict(), f'model/hasyv2_model_epoch_{epoch + 1}.pth')
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), 'model2/hasyv2_model_arg2.pth')
            print(f"✅ Сохранена лучшая модель с точностью {accuracy:.4f}")

    print(f"✅ Обучение завершено. Лучшая точность: {best_accuracy:.4f}, Лучшая Top-5 точность: {top5_accuracy:.4f}")

if __name__ == "__main__":
    main()

Нормализованный диапазон symbol_id: [0, 368]
Количество уникальных классов: 369
✅ Словарь сохранён в model/class_to_symbol_normalized.json
✅ Тестовые изображения сохранены в test_formatted
✅ Данные загружены: 134586 обучающих, 33647 тестовых образцов


/home/enovo/hasy-env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


✅ Модель на cuda
✅ Загружена предобученная модель из model/hasyv2_model_arg.pth


Epoch 1/50: 100%|███████████████████████████████████████████████████| 1052/1052 [00:36<00:00, 28.63batch/s, loss=1.8915]


Epoch [1/50], Average Loss: 1.8119


...

Точность на тестовой выборке: 0.7242 (24367/33647), Top-5 точность: 0.9673


Epoch 50/50: 100%|██████████████████████████████████████████████████| 1052/1052 [00:25<00:00, 40.85batch/s, loss=1.3875]


Epoch [50/50], Average Loss: 1.3444


Evaluating: 100%|██████████████████████████████████████████████████████████████████| 263/263 [00:02<00:00, 97.47batch/s]

Точность на тестовой выборке: 0.7251 (24396/33647), Top-5 точность: 0.9687
✅ Обучение завершено. Лучшая точность: 0.7308, Лучшая Top-5 точность: 0.9687
